# Quick Start

Below is a simple demo of interaction with the environment of the vm sheduling sceanrio.

In [4]:
from maro.simulator import Env
from maro.simulator.scenarios.vm_scheduling import PlaceAction, DecisionPayload

env = Env(scenario="vm_scheduling", topology="azure.2019.10k", start_tick=0, durations=8638, snapshot_resolution=1)

metrics: object = None
decision_payload: DecisionPayload = None
is_done: bool = False

while not is_done:
    action: PlaceAction = None
    metrics, decision_payload, is_done = env.step(action)

print(metrics)

Download finished.
Unzip /home/kuanweiyu/.maro/data/vm_scheduling/.build/azure.2019.10k/azure.2019.10k.tar.gz to /home/kuanweiyu/.maro/data/vm_scheduling/.build/azure.2019.10k
{'total_vm_requests': 10000, 'total_energy_consumption': 863800.0, 'successful_placement': 0, 'successful_completion': 0, 'failed_placement': 0, 'total_latency': Latency(Agent=0, Resource=0), 'total_oversubscriptions': 0}


# Environment of the vm scheduling

To initialize an environment, you need to specify the values of several parameters:

- **scenario**: The target scenario of this Env.
  - vm_scheduling denotes for the virtual machine scheuling.
- **topology**: The target topology of this Env. As shown below, you can get the predefined topology list by calling `get_topologies(scenario='vm_scheduling')`
- **start_tick**: The start tick of this Env, 1 tick corresponds to 5 minutes in vm_scheduling.
  - In the demo above, start_tick=0 indicates a simulation start from the beginning of the given topology.
- **durations**: The duration of thie Env, in the unit of tick/5minute.
  - In the demo above, durations=8638 indicates a simulation length of 30 days (30d * 24h * 60m / 5).
- **snapshot_resolution**: The time granularity of maintaining the snapshots of the environments, in the unit of tick/5minute.
  - In the demo above, snapshot_resolution=1 indicates that a snapshot will be created and saved every tick during the simulation.
  
You can get all available scenarios and topologies by calling:


In [6]:
from maro.simulator.utils import get_scenarios, get_topologies
from pprint import pprint
from typing import List

scenarios: List[str] = get_scenarios()
topologies: List[str] = get_topologies(scenario='vm_scheduling')

pprint(f'The available scenarios in MARO:')
pprint(scenarios)

print()
pprint(f'The predefined topologies in VM Scheuling:')
pprint(topologies)

'The available scenarios in MARO:'
['cim', 'citi_bike', 'vm_scheduling']

'The predefined topologies in VM Scheuling:'
['azure.2019.10k', 'azure.2019.336k', 'azure.2019.original']


Once you created an instance of the environment, you can easily access the real-time information of this environment, like:

In [8]:
from maro.backends.frame import SnapshotList
from maro.simulator import Env
from pprint import pprint
from typing import List


# Initialize an Env for citi_bike scenario
env = Env(scenario="vm_scheduling", topology="azure.2019.10k", start_tick=0, durations=8638, snapshot_resolution=1)

# The current tick
tick: int = env.tick
print(f"The current tick: {tick}.")

# The current frame index, which indicates the index of current frame in the snapshot-list
frame_index: int = env.frame_index
print(f"The current frame index: {frame_index}.")

# The whole snapshot-list of the environment, snapshots are taken in the granularity of the given snapshot_resolution
# The example of how to use the snapshot will be shown later
snapshot_list: SnapshotList = env.snapshot_list
print(f"There will be {len(snapshot_list)} snapshots in total.")

# The summary info of the environment
summary: dict = env.summary
print(f"\nEnv Summary:")
pprint(summary)

# The metrics of the environment
metrics: dict = env.metrics
print(f"\nEnv Metrics:")
pprint(metrics)

The current tick: 0.
The current frame index: 0.
There will be 8638 snapshots in total.

Env Summary:
{'event_payload': {'PENDING_DECISION': ['valid_pms',
                                        'vm_id',
                                        'vm_cpu_cores_requirement',
                                        'vm_memory_requirement',
                                        'remaining_buffer_time'],
                   'REQUEST': ['vm_info', 'remaining_buffer_time']},
 'node_detail': {'pms': {'attributes': {'cpu_cores_allocated': {'slots': 1,
                                                                'type': 'i2'},
                                        'cpu_cores_capacity': {'slots': 1,
                                                               'type': 'i2'},
                                        'cpu_utilization': {'slots': 1,
                                                            'type': 'f'},
                                        'energy_consumption': {'slots': 1,

# Interaction with the environment

Before starting interaction with the environment, we need to know **DecisionPayload** and **Action** first.